In [ ]:
#RNA velocity analysis of muscle mimetic cells

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy as scp
import sklearn
import matplotlib.pyplot as plt
import matplotlib
import sys
import glob
import pickle
from collections import Counter
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
import loompy
import scipy.optimize
import packaging
import scanpy as sc
import scvelo as scv
import anndata
print(packaging.__version__) 
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import cellrank as cr


In [ ]:
date = '20240110'

In [ ]:
scv.settings.set_figure_params('scvelo')
plt.rcParams['figure.dpi'] = 600

In [ ]:
## Import data from 5 donors and merge with original cluster labels

#import RNA velocity count matrices 
directory = '/n/groups/cbdm_lab/brh040/analysis/rna_velocity/human_HT2-HT6/preprocessing/kb_count_out_ht2/counts_unfiltered/'
adata_ht2 = scv.read(directory+'adata.h5ad')

#import RNA velocity count matrices 
directory = '/n/groups/cbdm_lab/brh040/analysis/rna_velocity/human_HT2-HT6/preprocessing/kb_count_out_ht3/counts_unfiltered/'
adata_ht3 = scv.read(directory+'adata.h5ad')

#import RNA velocity count matrices 
directory = '/n/groups/cbdm_lab/brh040/analysis/rna_velocity/human_HT2-HT6/preprocessing/kb_count_out_ht4/counts_unfiltered/'
adata_ht4 = scv.read(directory+'adata.h5ad')

#import RNA velocity count matrices 
directory = '/n/groups/cbdm_lab/brh040/analysis/rna_velocity/human_HT2-HT6/preprocessing/kb_count_out_ht5/counts_unfiltered/'
adata_ht5 = scv.read(directory+'adata.h5ad')

#import RNA velocity count matrices 
directory = '/n/groups/cbdm_lab/brh040/analysis/rna_velocity/human_HT2-HT6/preprocessing/kb_count_out_ht6/counts_unfiltered/'
adata_ht6 = scv.read(directory+'adata.h5ad')

In [ ]:
# load cell metadata:
directory = '/home/brh040/analysis/rna_velocity/human_HT2-HT6/seurat_output-from_local/'
cell_meta = pd.read_csv(directory+"20240110_metadata_muscle.csv")
cell_meta.index = cell_meta['barcode'] #critical step so merge happens -- needs barcodes as index

#in seurat data file, barcodes end with '2' if from HT3 and in '1' if from HT2

#relabel barcodes in adata to match this convention
adata_ht2.obs.index = adata_ht2.obs.index + '-1_1'
adata_ht3.obs.index = adata_ht3.obs.index + '-1_2'
adata_ht4.obs.index = adata_ht4.obs.index + '-1_3'
adata_ht5.obs.index = adata_ht5.obs.index + '-1_4'
adata_ht6.obs.index = adata_ht6.obs.index + '-1_5'

In [ ]:
#merge metadata with adata object
adata_ht2.obs = pd.merge(left=adata_ht2.obs, right=cell_meta['clusterLabels'], left_index=True, right_index=True,how='left')
adata_ht3.obs = pd.merge(left=adata_ht3.obs, right=cell_meta['clusterLabels'], left_index=True, right_index=True,how='left')
adata_ht4.obs = pd.merge(left=adata_ht4.obs, right=cell_meta['clusterLabels'], left_index=True, right_index=True,how='left')
adata_ht5.obs = pd.merge(left=adata_ht5.obs, right=cell_meta['clusterLabels'], left_index=True, right_index=True,how='left')
adata_ht6.obs = pd.merge(left=adata_ht6.obs, right=cell_meta['clusterLabels'], left_index=True, right_index=True,how='left')


In [ ]:
adata_ht2.layers['unspliced']

In [ ]:
adata_ht2.layers['spliced']

In [ ]:
cell_meta

In [ ]:
print(len(adata_ht2.obs[~adata_ht2.obs['clusterLabels'].isnull()]))
print(len(adata_ht3.obs[~adata_ht3.obs['clusterLabels'].isnull()]))
print(len(adata_ht4.obs[~adata_ht4.obs['clusterLabels'].isnull()]))
print(len(adata_ht5.obs[~adata_ht5.obs['clusterLabels'].isnull()]))
print(len(adata_ht6.obs[~adata_ht6.obs['clusterLabels'].isnull()]))

In [ ]:
pd.unique(adata_ht2.obs['clusterLabels'])

In [ ]:
def subset(adata):
    idx = ( adata.obs['clusterLabels']=='Early') | ( adata.obs['clusterLabels']=='Inter1') | \
    ( adata.obs['clusterLabels']=='Inter2') | ( adata.obs['clusterLabels']=='Late') | \
    ( adata.obs['clusterLabels']=='lncRNA-enriched') 
    return idx

idx_ht2 = subset(adata_ht2)
idx_ht3 = subset(adata_ht3)
idx_ht4 = subset(adata_ht4)
idx_ht5 = subset(adata_ht5)
idx_ht6 = subset(adata_ht6)

subset_adata_ht2 = adata_ht2[idx_ht2].copy()
subset_adata_ht3 = adata_ht3[idx_ht3].copy()
subset_adata_ht4 = adata_ht4[idx_ht4].copy()
subset_adata_ht5 = adata_ht5[idx_ht5].copy()
subset_adata_ht6 = adata_ht6[idx_ht6].copy()

In [ ]:
subset_adata_ht2.layers['spliced']

In [ ]:
cell_meta_ht2 = cell_meta[cell_meta['barcode'].str[-1]=='1'] #for HT2
cell_meta_ht3 = cell_meta[cell_meta['barcode'].str[-1]=='2'] #for HT3
cell_meta_ht4 = cell_meta[cell_meta['barcode'].str[-1]=='3'] #for HT4
cell_meta_ht5 = cell_meta[cell_meta['barcode'].str[-1]=='4'] #for HT5
cell_meta_ht6 = cell_meta[cell_meta['barcode'].str[-1]=='5'] #for HT6

subset_adata_ht2.obsm['X_umap'] = np.vstack((cell_meta_ht2['UMAP_1'].to_numpy(), cell_meta_ht2['UMAP_2'].to_numpy())).T #original umap coordinates
subset_adata_ht3.obsm['X_umap'] = np.vstack((cell_meta_ht3['UMAP_1'].to_numpy(), cell_meta_ht3['UMAP_2'].to_numpy())).T #original umap coordinates
subset_adata_ht4.obsm['X_umap'] = np.vstack((cell_meta_ht4['UMAP_1'].to_numpy(), cell_meta_ht4['UMAP_2'].to_numpy())).T #original umap coordinates
subset_adata_ht5.obsm['X_umap'] = np.vstack((cell_meta_ht5['UMAP_1'].to_numpy(), cell_meta_ht5['UMAP_2'].to_numpy())).T #original umap coordinates
subset_adata_ht6.obsm['X_umap'] = np.vstack((cell_meta_ht6['UMAP_1'].to_numpy(), cell_meta_ht6['UMAP_2'].to_numpy())).T #original umap coordinates


In [ ]:
#concatenate
adata = subset_adata_ht2.concatenate(subset_adata_ht3,subset_adata_ht4)
adata = adata.concatenate(subset_adata_ht5,subset_adata_ht6)

In [ ]:
######### Calculate and plot UMAP 
# pre-process data
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=20000) 
scv.pp.moments(adata)

In [ ]:
adata.layers['spliced']

In [ ]:
### Plot Maps
sc.pl.umap(adata, color=['clusterLabels'], frameon=False, save=date+'_muscle_umap.pdf')

In [ ]:
mode = 'dynamical'
donori='ht_all'
umapi='orig'
clusteri='-muscle'

scv.tl.recover_dynamics(adata, n_jobs=3)
scv.tl.velocity(adata, mode='dynamical')
#continued in next cell...

In [ ]:
# ... continued from prior cell
scv.tl.velocity_graph(adata, n_jobs=3)
scv.pl.velocity_embedding_stream(adata, basis='umap', color='clusterLabels',title='', 
                                 save=date+'_embedding_'+mode+'_stream_'+donori+'_umap_'+umapi+clusteri+'.png', 
                                 legend_loc='right margin')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color='clusterLabels',
                                 title='', 
                                 save=date+'_embedding_'+mode+'_stream_'+donori+'_umap_'+umapi+clusteri+'.png', 
                                 legend_loc='right margin',alpha=0.3,
                                 palette=["#4E79A7","#F28E2B",  "#59A14F", "#B07AA1", "#FABFD2"],
                                 figsize=(4.5,5.5),
                                size=15)

In [ ]:
scv.tl.velocity_confidence(adata)
scv.pl.scatter(adata, color='velocity_confidence', perc=[2,98])

In [ ]:
#save confidence image
scv.pl.scatter(adata, color='velocity_confidence', perc=[2,98],
               title='', 
               save=date+'_velocity-confidence_'+donori+'_umap_'+umapi+clusteri+'.png', 
               legend_loc='right margin',
               figsize=(4.5,5.5),
               size=15)
              

In [ ]:
# save 
adata.write(date+'_adata_ht-muscle.h5ad')

In [ ]:
#cell rank
cr.tl.terminal_states(adata, cluster_key="clusterLabels", weight_connectivities=0.2)
cr.pl.terminal_states(adata, save=date+'_cr_terminalstates-muscle.pdf')

In [ ]:
cr.tl.initial_states(adata, cluster_key="clusterLabels")
cr.pl.initial_states(adata, discrete=True, save=date+'_cr_initialstates-muscle.pdf')

In [ ]:
cr.tl.lineages(adata)

In [ ]:
cr.pl.lineages(adata, same_plot=False, save=date+'_cr_lineages-muscle.pdf',
              figsize=(4.5,5.5))

In [ ]:
cr.pl.lineages(adata, same_plot=True , save=date+'_cr_lineages-sameplot-colors-muscle.pdf',
               color=['black','red'],
              figsize=(5,5.5))


In [ ]:
scv.tl.recover_latent_time(
    adata, root_key="initial_states_probs", end_key="terminal_states_probs"
)

In [ ]:
scv.tl.paga(
    adata,
    groups="clusterLabels",
    root_key="initial_states_probs",
    end_key="terminal_states_probs",
    use_time_prior="velocity_pseudotime",
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="paga_pie",
    cluster_key="clusterLabels",
    basis="umap",
    legend_kwargs={"loc": "top right out"},
    legend_loc="top left out",
    node_size_scale=5,
    edge_width_scale=1,
    max_edge_width=4,
    title="directed PAGA",save=date+'_cr_paga-muscle.pdf',
    figsize=(4.5,5)
)


In [ ]:
#https://cellrank.readthedocs.io/en/stable/cellrank_basics.html